In [1]:
#Importing importing Files

import re
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [126]:
#Reading File Data

fileData=[]
with open("Output.txt", "r") as file:
    for line in file:
        x = re.sub("=", " = ", line)
        x = re.sub("  "," ", x)   
        fileData.append(x)

In [127]:
#Loading Shelf Data from "Data Generator"
import shelve
import os

dest_file="NLP_Shelfv11//" + "Dict_Lists" + ".shlf"

shelf = shelve.open(dest_file)

dataType_number = shelf["dataType_number"]
dataType_char = shelf["dataType_char"]
dataType_list = shelf["dataType_list"]
dataType_bool = shelf["dataType_bool"]
dataType_class = shelf["dataType_class"]
dataType_void = shelf["dataType_void"]
dataType = shelf["dataType"]

variables_list = shelf["variables_list"]
func_name = shelf["func_name"]

scope_of_variable = shelf["scope_of_variable"]

operators = shelf["operators"]

command_declare = shelf["command_declare"]
command_print = shelf["command_print"]
command_input = shelf["command_input"]

func_prog = shelf["func_prog"]

special_class = shelf["special_class"]
special_function = shelf["special_function"]

other_tags = shelf["other_tags"]
seperators = shelf["seperators"] 
equal_symbol = shelf["equal_symbol"]


shelf.close()

#print(dataType_number, "\n")
#print(dataType, "\n")
#print(variables_list, "\n")
#print(scope_of_variable, "\n")
#print(operators, "\n")
#print(command_declare, "\n")
#print(command_print, "\n")
#print(command_input, "\n")
#print(func_prog, "\n")
#print(special_class, "\n")
#print(special_function, "\n")

In [128]:
#Defining Patterns
#{"IN":["","s"]}
matcher = Matcher(nlp.vocab)

pattern1_1 = [{"LOWER":{"IN":dataType}}]
matcher.add("DATATYPE", None, pattern1_1)

pattern2_1 = [{"LOWER":"global"}]
pattern2_2 = [{"LOWER":"local"}]
matcher.add("SCOPE", None, pattern2_1, pattern2_2)

pattern3_1 = [{"LOWER":{"IN":operators}}]
pattern3_2 = [{"LOWER":{"IN":["additions", "adds"]}}]
pattern3_3 = [{"LOWER":{"IN":["subtractions", "subs"]}}]
pattern3_4 = [{"LOWER":{"IN":["minuses", "multiplies", "muls"]}}]
pattern3_5 = [{"LOWER":{"IN":["divides", "divs"]}}]
pattern3_6 = [{"LOWER":{"IN":["modulus", "moduluses", "mods"]}}]
matcher.add("OPERATOR", None, pattern3_1, pattern3_2, pattern3_3, pattern3_4, pattern3_5, pattern3_6)

pattern4_1 = [{"LOWER":{"IN":command_declare}}]
pattern4_2 = [{"LOWER":{"IN":command_print}}]
pattern4_3 = [{"LOWER":{"IN":command_input}}]
pattern4_4 = [{"LOWER":{"IN":special_function}}]
matcher.add("COMMAND", None, pattern4_1, pattern4_2, pattern4_3, pattern4_4)

pattern5_1 = [{"LOWER":{"IN":["function","functions"]}}]
pattern5_2 = [{"LOWER":{"IN":["func","funcs"]}}]
pattern5_3 = [{"LOWER":{"IN":["program","programs"]}}]
pattern5_4 = [{"LOWER":{"IN":["programme","programmes"]}}]
pattern5_5 = [{"LOWER":{"IN":["prog","progs"]}}]

pattern5_1 = [{"LOWER":{"IN":func_prog}}]
pattern5_2 = [{"LOWER":{"IN":["funcs","functions"]}}]
pattern5_3 = [{"LOWER":{"IN":["programs","programmes","progs"]}}]
matcher.add("FUNC_PROG", None, pattern5_1, pattern5_2, pattern5_3)

pattern6_1 = [{"LOWER": {"IN": variables_list}}]
matcher.add("VARI", None, pattern6_1)

pattern7_1 = [{"LOWER": {"IN": special_class}}]
matcher.add("SPL_CLASS", None, pattern7_1)

pattern8_1 = [{"LIKE_NUM": True}]
matcher.add("VALUE", None, pattern8_1)

pattern9_1 = [{"LOWER": {"IN": seperators}}]
matcher.add("SEPERATOR", None, pattern9_1)

pattern10_1 = [{"LOWER":{"IN":equal_symbol}}]
matcher.add("EQUAL", None, pattern10_1)

pattern11_1 = [{"ORTH": "'"}, {"TEXT": {"REGEX": "."}, "OP":"*"},{"ORTH": "'"}]
pattern11_2 = [{"ORTH": "\""}, {"TEXT": {"REGEX": "."}, "OP":"*"},{"ORTH": "\""}]
matcher.add("TEXT", None, pattern11_1, pattern11_2)

#pattern12_1 = [{"ORTH": "two"}]
#pattern12_2 = [{"ORTH": "3"}]
#matcher.add("COUNT", None, pattern12_1, pattern12_2)

#pattern12_1 = [{"LOWER": {"IN": other_tags}}]
#matcher.add("OTHER", None, pattern12_1)

In [129]:
#Creating Training Data

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(fileData):
    
    #x = re.sub("=", " = ", doc)
    #doc = re.sub("  "," ", x)
    
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    #print(spans)
    
    #Modified Start
    entity_label=[]
    entity_label = [(nlp.vocab.strings[match_id]) for match_id, start, end in matcher(doc)]
    #print(entity_label)
    
    #Remove duplicates
    #if len(spans) != len(set(spans))
    

    entities = [(span.start_char, span.end_char, entity_label[index]) for index,span in enumerate(spans)]
    #print(entities)
    #tags = [(entity_label[index]) for index,span in enumerate(spans)]
    
    #for eachWord in entities:
    #    print("Entities: {}".format(eachWord))
    
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    #training_example = (doc.text, {"entities": entities}, {"tags": tags})
    
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    #Modified End
    
    '''
    entities = [(span.start_char, span.end_char, "INITIALIZATION") for span in spans]
    
    for eachWord in entities:
        print("Entities: {}".format(eachWord))
        
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    '''
    

#print(*TRAINING_DATA, sep="\n")

In [130]:
#Store to Data file for inspection
with open("training_data.txt","w") as o_file:
    for item in TRAINING_DATA:
        o_file.write("{} \n".format(item))

In [32]:
#Store Pickle Data
import pickle

with open('training_data.pkl', 'wb') as f:
    pickle.dump(TRAINING_DATA, f)

In [27]:
#Load Pickle data
with open('training_data.pkl', 'rb') as f:
    TRAINING_DATA = pickle.load(f)

In [131]:
#Training Loop
import random

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

#tagger = nlp.create_pipe("tagger")
#nlp.add_pipe(tagger) 


ner.add_label("DATATYPE")
ner.add_label("SCOPE")
ner.add_label("OPERATOR")
ner.add_label("COMMAND")
ner.add_label("FUNC_PROG")
ner.add_label("VARI")
ner.add_label("SPL_CLASS")
ner.add_label("VALUE")
ner.add_label("COUNT")
ner.add_label("SEPERATOR")
ner.add_label("EQUAL")
ner.add_label("TEXT")
#ner.add_label("OTHER")

'''
tagger.add_label("DATATYPE")
tagger.add_label("SCOPE")
tagger.add_label("OPERATOR")
tagger.add_label("COMMAND")
tagger.add_label("FUNC_PROG")
tagger.add_label("VARI")
tagger.add_label("SPL_CLASS")
tagger.add_label("VALUE")
tagger.add_label("COUNT")
tagger.add_label("SEPERATOR")
tagger.add_label("EQUAL")
tagger.add_label("TEXT")
tagger.add_label("OTHER")
'''

print(nlp.pipe_names)

# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(3):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=1000):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]
        #for text, entities, tags in batch:
        #    annotations.append(entities)   
        #    annotations.append(tags)
        
        #print(texts)
        #print(annotations)

        # Update the model
        nlp.update(texts, annotations, losses=losses)
               
        #annotations_tag = [tags for text, entities, tags in batch]
        #print(annotations_tag)
        #nlp.update(texts, annotations_tag, losses=losses)
        
    print(losses)

['ner']


ValueError: [E103] Trying to set conflicting doc.ents: '(37, 42, 'EQUAL')' and '(37, 45, 'EQUAL')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.

In [110]:
#NER Test
from spacy import displacy

#doc = nlp("create a function to add numbers z and b")
#doc = nlp("if a > b then add a and b")
#doc = nlp("get input k")
#doc = nlp("if a>b then print a. If b>c then print c")

doc = ("display variable text1 equals to 'some thing something hello there. General Kenobi' on screen")
#doc = re.sub("=", " = ", doc)
#doc = re.sub("  "," ", doc) 

doc = nlp(doc)
  

for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

display 0 7 COMMAND
variable 8 16 DATATYPE
text1 17 22 VARI
equals 23 29 VARI
'some thing something hello there. General Kenobi' 33 83 TEXT


In [20]:
# Tagger Test

test_text = "Write a program for mod doubles b = 77 and c = doubles 77"
doc = nlp(test_text)
print("Tags", [(t.text, t.tag_, t.pos_) for t in doc])

Tags [('Write', '""', 'PUNCT'), ('a', '""', 'PUNCT'), ('program', '""', 'PUNCT'), ('for', '""', 'PUNCT'), ('mod', '""', 'PUNCT'), ('doubles', '""', 'PUNCT'), ('b', '""', 'PUNCT'), ('=', '""', 'PUNCT'), ('77', '""', 'PUNCT'), ('and', '""', 'PUNCT'), ('c', '""', 'PUNCT'), ('=', '""', 'PUNCT'), ('doubles', '""', 'PUNCT'), ('77', '""', 'PUNCT')]


In [111]:
#Saving the nlp Object
nlp.to_disk("./NLP_Object")

In [ ]:
#Importing the nlp Object
import spacy
from spacy import displacy
nlp = spacy.load("./NLP_Object")

doc = nlp("create a function to add numbers a and b")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)